# Annotation of genomes

## Introduction to annotation tools

Once we have analysis ready genomes (verified quality and lack of contamination), we can do annotation. The main goal of annotation is to identify the genomic features that are present in the contigs. There are many available tools to find these relevant sequences, and they mostly use predictive algorithms (recognition of common sequences in the DNA) or alignment against a reference database. 

Two of the most commonly used tools are [BAKTA](https://github.com/oschwengers/bakta)  (https://doi.org/10.1099/mgen.0.000685) and the [NCBI Prokaryotic Genome Annotation Pipeline](https://github.com/ncbi/pgap) (https://doi.org/10.1093/nar/gkaa1105). 

Bakta produces a comprehensive annotation of Open Reading Frames (sequences that code for amino acids over a length threshold), different types of RNA (ncRNA, tRNA, rRNA), hypothetical proteins, genes and CRISPR arrays. 

In [1]:
# source PATH to use module function
source /cvmfs/soft.computecanada.ca/config/profile/bash.sh

Bakta uses a reference database that is to large for a tutorial (~30GB), so we are using a reduced database (~3.0 GB) that may produce less reliable feature annotation. 

The database was downloaded and decompressed into `/tools/reference_data/db-light` using the command below
```
singularity exec -B /scratch,/project,/etc </PATH/TO/BAKTA/IMAGE/> \
    bakta_db download \
    --output </OUTPUT/DIR/> \
    --type light
```

## Annotation of draft genomes

First, we can review the structure of our working directory. 

### **Note:** Remember to have **bash_kernel** for the Jupyter notebook loaded.

In [3]:
# review the project structure
tree -dL 2 ~/tutorials

# software directory structure
tree -dL 2 /mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/ 

/home/jupyter-mdprieto/tutorials
|-- results
|   |-- annotation
|   |-- assembly_checkm
|   |-- assembly_quast
|   |-- contigs
|   |-- reads_qc
|   `-- snippy
`-- trimmed_reads

8 directories
/mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/
|-- baktadb-light
|   `-- amrfinderplus-db
|-- raw_reads
`-- reference_data

4 directories


We save the path to the **bakta singularity image** and the **reference database** in the following [ENVIRONMENTAL VARIABLES](https://linuxize.com/post/how-to-set-and-list-environment-variables-in-linux/) to use them in our commands

In [5]:
BAKTA_IMG="/mnt/cidgoh-object-storage/images/bakta_1.7.sif"
BAKTA_DB="/mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/baktadb-light"

Now, we execute the command `bakta` specifying a single draft genome. Annotation for the remaining ones is already provided (7 min per annotation).

In [9]:
# remove previously produced results
rm ~/tutorials/results/annotation/ERR10479518*

# add drives so singularity can look inside them
export SINGULARITY_BIND="/mnt,/etc"

In [6]:
singularity exec -B /etc "$BAKTA_IMG" bakta \
    --db $BAKTA_DB                                                                  `# path to bakta database` \
    ~/tutorials/results/contigs/ERR10479518_contigs.fa                              `# file to annotate`\
    --output ~/tutorials/results/annotation/                                        `# output directory` \
    --genus Pseudomonas                                                             `# specify genus of isolate` \
    --prefix $(echo "ERR10479518_contigs.fa" | grep -Eo "ERR[0-9]+")                `# prefix of sample name only for output`


parse genome sequences...
	imported: 142
	filtered & revised: 142
	contigs: 142

start annotation...
predict tRNAs...
Traceback (most recent call last):
  File "/usr/local/bin/bakta", line 10, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.10/site-packages/bakta/main.py", line 153, in main
    genome['features'][bc.FEATURE_T_RNA] = t_rna.predict_t_rnas(genome, contigs_path)
  File "/usr/local/lib/python3.10/site-packages/bakta/features/t_rna.py", line 59, in predict_t_rnas
    proc = sp.run(
  File "/usr/local/lib/python3.10/subprocess.py", line 505, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/local/lib/python3.10/subprocess.py", line 1154, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/local/lib/python3.10/subprocess.py", line 2005, in _communicate
    ready = selector.select(timeout)
  File "/usr/local/lib/python3.10/selectors.py", line 416, in select
    fd_event_list = self._sele

Now, our analysis results look something like this. The directory structure now includes results from annotation and the tools directory includes a reference database to run bakta. 

In [7]:
# project structure
tree -dL 2 ~/tutorials

/home/jupyter-mdprieto/tutorials
|-- results
|   |-- annotation
|   |-- assembly_checkm
|   |-- assembly_quast
|   |-- contigs
|   |-- reads_qc
|   `-- snippy
`-- trimmed_reads

8 directories


The output of **Bakta** annotation can be found in `tutorials/results/annotation`. 

- Circular plots can be visualized directly on Jupyter by clicking them or downloaded to your local computer and opened.
- Annotation results in this dataset are not ideal as we are working with draft genomes instead of complete chromosome asssemblies or scaffolds. These more refined assemblies require long reads to polish the contigs.

Take a look at this paper for more information about assembly approaches: https://doi.org/10.1093/bib/bbw096. 

# Phylogenetic tree

To produce a phylogenetic tree we must compare the genomes using some kind of metric. Here, we will use **Snippy** to identify single nucleotide variants (SNV) in our genomes (https://github.com/tseemann/snippy). 

**Gubbins** is a tool that removes genes or features that were obtained through horizontal gene transfer and that may obscure the interpretation of the phylogenetic tree (https://github.com/nickjcroucher/gubbins). 

In [ ]:
SNIPPY_IMG="/mnt/cidgoh-object-storage/images/snippy_4.6.0.sif"
GUBBINS_IMG="/mnt/cidgoh-object-storage/images/gubbins-3.2.1.img"
REF_GENOME="/mnt/cidgoh-object-storage/seagull/jupyter-mdprieto/reference_data/GCF_000006765.1_ASM676v1_genomic.fna"

**Snippy** is a relatively fast tool, so we can analyze all the genomes we have available without too much wait. We apply the tool to every available draft genome. 

- The recommended use for the tool is actually on raw reads. 

To avoid typing the same command for every draft genome, we use a [**_for loop_**](https://ryanstutorials.net/bash-scripting-tutorial/bash-loops.php) to quickly reproduce the same command for a list of contig files. 

In [10]:
for CONTIG in $(ls $HOME/tutorials/results/contigs/*contigs.fa)
do
    
    # define isolate name
    SAMPLE_PREFIX=$(basename "$CONTIG" '_contigs.fa')
    echo "Processing $SAMPLE_PREFIX"

    # run snippy for each isolate
    singularity exec $SNIPPY_IMG snippy \
    --outdir $HOME/tutorials/results/snippy/$SAMPLE_PREFIX  `# save in a subdirectory named after sample ID` \
    --ctgs "$CONTIG" \
    --ref $REF_GENOME \
    --cpus 8 \
    --force \
    --cleanup \
    --quiet

done

# produce core genome analysis in results_snippy folder

cd $HOME/tutorials/results/snippy/
singularity exec "$SNIPPY_IMG" snippy-core \
    --ref $REF_GENOME \
    $HOME/tutorials/results/snippy/ERR*

Processing ERR10479510
[M::bwa_idx_load_from_disk] read 0 ALT contigs
[samclip] samclip 0.4.0 by Torsten Seemann (@torstenseemann)
[samclip] Loading: reference/ref.fa.fai
[samclip] Found 1 sequences in reference/ref.fa.fai
[M::process] read 320000 sequences (80000000 bp)...
[M::process] read 253774 sequences (63420526 bp)...
[M::mem_process_seqs] Processed 320000 reads in 33.555 CPU sec, 4.139 real sec
[samclip] Processed 100000 records...
[samclip] Processed 200000 records...
[samclip] Processed 300000 records...
[M::mem_process_seqs] Processed 253774 reads in 28.675 CPU sec, 3.523 real sec
[samclip] Processed 400000 records...
[samclip] Processed 500000 records...
[main] Version: 0.7.17-r1188
[main] CMD: bwa mem[samclip] Total SAM records 574968, removed 9431, allowed 1610, passed 565537
 -Y[samclip] Header contained 3 lines
 -M[samclip] Done.
 -R @RG\tID:ERR10479510\tSM:ERR10479510 -t 8 reference/ref.fa fake_reads.fq
[main] Real time: 9.468 sec; CPU: 63.437 sec
[bam_sort_core] mergi

Once we have the data from the previous command, we perform a few additional steps to clean our results and have them ready for phylogenetic analysis. 

1. We run **snippy-core** to summarize the **SNVs** that are present only in genes contained in all isolates

2. Then, **Gubbins** eliminates the features that are transmitted through horizontal gene transfer. 

3. Once we have the cleaned alignment, we use **snp-sites** to extract the sites in the alignment with a **SNV**

In [1]:
cd $HOME/tutorials/results/snippy

# clean file
singularity exec $SNIPPY_IMG snippy-clean_full_aln core.full.aln > clean.full.aln

# gubbins remove horizontal gene transfer 
singularity exec $GUBBINS_IMG run_gubbins.py -p gubbins clean.full.aln

# SNP-sites
singularity exec $SNIPPY_IMG snp-sites -c gubbins.filtered_polymorphic_sites.fasta > clean.core.aln

FATAL:   could not open image /home/jupyter-mdprieto/tutorials/results/snippy/snippy-clean_full_aln: failed to retrieve path for /home/jupyter-mdprieto/tutorials/results/snippy/snippy-clean_full_aln: lstat /home/jupyter-mdprieto/tutorials/results/snippy/snippy-clean_full_aln: no such file or directory
FATAL:   could not open image /home/jupyter-mdprieto/tutorials/results/snippy/run_gubbins.py: failed to retrieve path for /home/jupyter-mdprieto/tutorials/results/snippy/run_gubbins.py: lstat /home/jupyter-mdprieto/tutorials/results/snippy/run_gubbins.py: no such file or directory
FATAL:   could not open image /home/jupyter-mdprieto/tutorials/results/snippy/snp-sites: failed to retrieve path for /home/jupyter-mdprieto/tutorials/results/snippy/snp-sites: lstat /home/jupyter-mdprieto/tutorials/results/snippy/snp-sites: no such file or directory


: 255

Once we have conducted all of the above steps. We have a file that contains the **SNVs** present in genes common to all isolates (core-genes) and that were not obtained by horizontal gene transfer. 

This file can be used to create a phylogenetic tree. 

In [6]:
# produce tree
$HOME/tutorials/tools/FastTree -gtr -nt clean.core.aln > clean.core.tree

FastTree Version 2.1.11 SSE3
Alignment: clean.core.aln
Nucleotide distances: Jukes-Cantor Joins: balanced Support: SH-like 1000
Search: Normal +NNI +SPR (2 rounds range 10) +ML-NNI opt-each=1
TopHits: 1.00*sqrtN close=default refresh=0.80
ML Model: Generalized Time-Reversible, CAT approximation with 20 rate categories
Initial topology in 0.05 seconds
Refining topology: 14 rounds ME-NNIs, 2 rounds ME-SPRs, 7 rounds ML-NNIs
Total branch-length 3.002 after 0.72 sec, 1 of 9 splits   
ML-NNI round 1: LogLk = -72517.017 NNIs 1 max delta 3.17 Time 1.28
GTR Frequencies: 0.2436 0.2580 0.2725 0.2259ep 12 of 12   
GTR rates(ac ag at cg ct gt) 1.1798 14.4592 0.8504 0.7559 53.9537 1.0000
Switched to using 20 rate categories (CAT approximation)15 of 20   
Rate categories were divided by 0.677 so that average rate = 1.0
CAT-based log-likelihoods may not be comparable across runs
Use -gamma for approximate but comparable Gamma(20) log-likelihoods
ML-NNI round 2: LogLk = -67208.532 NNIs 1 max delta 0.0

Finally, we can visualize the tree we just produce by pasting the contents of the file `clean.core.tree` in a visualizer tool such as [phyl.io](https://phylo.io/), or [iTOL](https://itol.embl.de/upload.cgi).

The data we used may not be the best representation of differences in a phylogenetic tree as it comes from a single center outbreak and the samples may be highly similar among them.